Time Series는 확률변수를 시간 순으로 모아 놓은 추계적 과정(Stochastic process)의 일종이다. 시계열의 패턴을 요약해 시간에 따른 상관관계, 추세, 계절성 등의 특성을 파악하거나 미래 시점을 예측하기 위해 활용한다.

# (1) Smoothing method

시계열 패턴 파악을 쉽게 하기 위해 smoothing method를 널리 활용한다. smoothing method 중에서는 **Moving-Average**와 **Exponential Smoothing**이 많이 사용된다.

### (A) Moving Average

##### (a) Horizontal pattern

$X_t = c + a_t, \quad a_t \sim N(0, \sigma_a^2)$

시간 t에서 시계열 값 $X_t$가 위와 같을 때, 시점 T에서 최근 N개의 데이터만 사용해 $c$를 추정하기로 했다. 이때 제곱합 Q를 최소로 하는 $\hat{c}$은 $\frac{\partial Q}{\partial c} = 0$의 해를 구하면 된다. 그 값은 아래와 같다

$$Min_{c} Q = \sum_{t = T-N+1}^T (X_t - c)^2$$  

$$\hat{c} = \frac{1}{N}\sum_{t = T-N+1}^T X_t$$

이 $\hat{c}$를 시점 $T$에서 $X_t$에 대한 span size $N$의 **단순이동평균(Simple moving average)** 이라 하며,  
$$M_T = \frac{1}{N}\sum_{t = T-N+1}^T X_t$$라고 표기한다.

$M_T$의 성질은 아래와 같다.  

$$E(M_T) = E(X_t) = c$$  
$$Var(M_T) = \frac{1}{N^2} (N\cdot Var(X_t)) = \frac{\sigma_a^2}{N}$$

즉, 단순이동평균은 $c$의 U.E.이고, 분산은 시계열 분산의 $\frac{1}{N}$이다. 이 N값은 최근 추세를 얼마나 반영할지에 따라 결정할 수 있다.

시점 $T$보다 한 시점 더 나아간 $M_{T+1}$은 아래와 같이 표시할 수 있다. 아래와 같이 계산하면 데이터가 엄청 많은 경우 이동 평균을 계산할 때 **계산량을 조금 줄일 수 있다.**  

$$M_{T+1} = M_T + \frac{X_{T+1} - X_{T-N+1}}{N}$$

시점 T+1의 예측값은 어떻게 구할 수 있을까? $f_{T,1}$은 조건부 기댓값 $E(X_{T+1}\mid X_T, \cdots) = c$로 계산된다. horizontal pattern이면 결국 시점에 관계 없이 모든 예측값이 동일하다.

즉, $\hat{f}_{T,k} = \hat{c} = M_T$이다. 이를 **k-step-ahead prediction value**라고 부른다.

예측 값에 대한 예측 오차는 $e_{T,1} = X_{T+1} - \hat{f}_{T,1} = X_{T+1} - M_T$이고,  

$$Var(e_{T,1}) = Var(X_{T+1} - M_T) = Var(X_{T+1}) + Var(M_T) = (1 + \frac{1}{N})\sigma_a^2$$, 조금 더 Naive하게 보면  
$$Var(X_{T+1} - X_T) = Var(X_{T+1}) + Var(X_T) = 2\sigma_a^2 \geq (1+\frac{1}{N})\sigma_a^2$$

즉, $N$이 클수록 예측오차 분산이 작아진다. 하지만 $N$이 너무 크면 단기적 변동을 반영하지 못한다.

##### (b) Linear trend

이번에는 $X_t$가 선형 추세를 갖는다고 해보자. 즉,  

$$X_t = c + bt + a_t$$

이 경우 $c$와 $b$를 추정하기 위해 **이중 이동 평균(double moving average)** 를 쓸 수 있다. 이는 $N$개의 단순이동평균에 다시 이동평균을 취해 구할 수 있다.   

$$M_T^{(2)} = \frac{1}{N} \sum_{i=T-N+1}^T M_i$$

c와 b에 대한 추정값을 얻어보자.  

$$E(M_T) = \frac{1}{N} \sum_{t=T-N+1}^T E(X_t) = \frac{1}{N} \sum_{t=T-N+1}^T E(c+bt+a_t) = c+bT-\frac{N-1}{2}b$$  
$$E({M_T}^{(2)}) = \frac{1}{N} \sum_{i=T-N+1}^T E(M_i) = \frac{1}{N} \sum_{i=T-N+1}^T E(c + bi - \frac{N-1}{2}b = c + bT - (N-1)b$$  

두 식을 연립해서 풀면  

$$c = 2E(M_T) - E({M_T}^{(2)}) - bT$$  
$$b = \frac{2}{N-1}(E(M_T) - E({M_T}^{(2)}))$$  

따라서, $c$와 $b$에 대한 추정치는 각각  

$$\hat{c} = 2E(M_T) - E({M_T}^{(2)}) - bT$$  
$$\hat{b} = \frac{2}{N-1} (E(M_T) - E({M_T}^{(2)}))$$

이때,  

$$E(X_T) - E(M_T) = \frac{N-1}{2}b$$  
$$E(M_T) - E({M_T}^{(2)}) = \frac{N-1}{2}b$$  

따라서 시점 $T$에 대한 예측값은  

$$\hat{f}_{T,1} = \hat{c} + \hat{b}(T+1) = 2M_T - M_T^{(2)} + \hat{b}$$,  
$$\hat{f}_{T,k} = \hat{c} + \hat{b}(T+k) = 2M_T - M_T^{(2)} + k\hat{b}, \quad k = 1, 2, \cdots$$

### (B) Exponential Smoothing

가용한 데이터 전체를 활용해 smoothing을 하는 방법이다. 대신 시간에 따라 가중치를 다르게 배정한다.

##### (a) Horizontal pattern

다시 horizontal pattern을 고려할 때, 다음의 SS를 최소화하는 $c$를 추정하고자 한다.  

$$\underset{c}{Min}Q = \sum_{t=1}^T \lambda^{T-t}(X_t - c)^2, \quad \lambda: 0과 1 사이의 가중치  $$ 

즉, 시점 T에는 weight 1을 주고, T-1에는 $\lambda$, 시점 T-2에는 $\lambda^2 \cdots$의 weight을 주어 과거로 갈수록 작은 weight을 부여한다. 이 경우 $\partial Q / \partial c = 0$으로부터 $c$값은 다음과 같이 추정된다.  

$$\hat{c} = \frac{1-\lambda}{1-\lambda^T}\sum_{t=1}^T \lambda^{T-t}X_t$$,  
$$\hat{c} = (1-\lambda)\sum_{i=0}^\infty \lambda^iX_{T-i} \quad If\;T\;is\;very\;large$$

이를 시점 $T$에서 시계열 $X_t$에 대한 **단순지수평활값(Simple Exponential Smoothing)** 또는 **지수가중이동평균(Exponentially Weighted Moving Average ; EWMA)** 이라 부르고, 기호로는  
$$S_T = (1-\lambda)\sum_{i=0}^\infty \lambda^iX_{T-i}$$로 표기한다.  

만약 $\alpha = 1-\lambda$로 정의하면, $S_T = \alpha X_T + (1-\alpha)S_{T-1}$로 정의할 수 있다. 이것이 갱신식이 된다.  

이 경우 k-step prediction value는  

$$\hat{f}_{T,k} = \hat{c} + \hat{b}(T+k) = 2S_T - S_T^{(2)} + k\hat{b}, \quad k = 1, 2, \cdots$$  
이 방법을 **Brown 이중지수평활법**이라고 부른다. 이때,  

$$\hat{b} = \frac{\alpha}{1-\alpha} (S_T - S_T^{(2)})$$  
$$S_T^{(2)} = \alpha S_T + (1-\alpha) S_{T-1}^{(2)}$$


##### (b) Other method

Brown 이중지수평활법 외에도 수준과 추세를 각각 갱신하는 **Holt의 선형추세 지수평활법**, 계절성을 추가 반영한 **Holt-Winters 모형** 등이 있다.

### (C) Decomposing method

일반적으로 시계열은 추세 $b_t$, 계절성 $s_t$, 랜덤 성분 $\varepsilon_t$ 3가지 성분으로 구성된다. 경기 변동 등의 cycle이 추가될 수도 있으나 시계열이 아주 길지 않는 한 생략할 수 있다. 이 3가지 성분이 가법적으로 결합된다는 가정에 의한 Additive model을 만들 수 있고, 3가지 성분이 승법적으로 결합된다는 가정 하에 multiplicative model을 만들 수 있다.  

다만 이런 방법은 추세, 순환 성분을 추정할 때 정보 손실이 발생하며, random 성분 추정 시 오차가 생긴다. 그리고 급등/급락 기간에 값이 크게 변화하며, 계절 성분이 주기별로 일정하지 않다면 사용할 수 없다. 따라서 단기간 데이터 분해에는 적합하지만, 장기간 데이터에는 다소 부적합하다.

#### (a) Additive model

$data - trend = seasonal + random$

random 요소를 바로 추정하지 않기에 random 성분 추정을 위해서는 trend와 seasonal 성분 하나씩 빼준다.

#### (b) multiplicative model

$\frac{data}{trend} = seasonal\cdot random$

마찬가지로 random 성분을 직접 추정하지 않고 trend와 seasonal 성분을 나누어 준다.

### (D) Metric of Evaluating Prediction error

Let $$e_{t,1} = X_{t+1} - f_{t,1}$$

$$MSE = \frac{1}{n}\sum_{t=1}^n e_{t,1}^2$$  
$$RMSE = \sqrt[2]{\frac{1}{n}\sum_{t=1}^n e_{t,1}^2}$$  
$$MAD = \frac{1}{n}\sum_{t=1}^n \left | e_{t,1} \right |$$  
$$MAPE = \frac{100}{n} \sum_{t=1}^n \left | \frac{e_{t,1}}{X_{t+1}} \right |$$